In [ ]:
import pandas as pd

df = pd.read_csv("glass-data.csv", sep="\t")
df.columns = df.columns.str.strip()  

# Đổi tên cột để dễ thực hiện
df.columns = [
    "Id", "RI", "Na", "Mg", "Al", "Si", "K", "Ca", "Ba", "Fe", "Type"
]

# a. Kích thước dữ liệu
print("a. Kích thước dữ liệu:", df.shape)
print("   Số hàng (mẫu):", df.shape[0])
print("   Số cột (thuộc tính):", df.shape[1])

# b. Kiểu dữ liệu
print("\nb. Kiểu dữ liệu của các thuộc tính:")
print(df.dtypes)

# c. Số lượng từng giá trị nhãn (cột Type)
print("\nc. Số lượng thực thể của từng giá trị nhãn:")
print(df["Type"].value_counts())

# d. Giá trị min, max, mean
print("\nd. Giá trị thống kê của các cột số thực:")
print(df.describe().loc[["min", "max", "mean"]])


a. Kích thước dữ liệu: (214, 11)
   Số hàng (mẫu): 214
   Số cột (thuộc tính): 11

b. Kiểu dữ liệu của các thuộc tính:
Id        int64
RI      float64
Na      float64
Mg      float64
Al      float64
Si      float64
K       float64
Ca      float64
Ba      float64
Fe      float64
Type      int64
dtype: object

c. Số lượng thực thể của từng giá trị nhãn:
Type
2    76
1    70
7    29
3    17
5    13
6     9
Name: count, dtype: int64

d. Giá trị thống kê của các cột số thực:
         Id        RI        Na        Mg        Al         Si         K  \
min     1.0  1.511150  10.73000  0.000000  0.290000  69.810000  0.000000   
max   214.0  1.533930  17.38000  4.490000  3.500000  75.410000  6.210000   
mean  107.5  1.518365  13.40785  2.684533  1.444907  72.650935  0.497056   

             Ca        Ba        Fe      Type  
min    5.430000  0.000000  0.000000  1.000000  
max   16.190000  3.150000  0.510000  7.000000  
mean   8.956963  0.175047  0.057009  2.780374  
